# Prompting

Prompting in LLMs is the design of a structured input to provide task description, demostrations and the actual input for the model to generate a desired output.

In [1]:
!pip install datasets evaluate transformers==4.30 accelerate peft bitsandbytes
!pip install sacrebleu
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: fsspec


In this notebook, we are going to use for fine-tuning a dataset set that is already available in the [Datasets repository](https://huggingface.co/datasets) from Hugging Face. However, the [Datasets library](https://huggingface.co/docs/datasets) makes easy to access and load datasets. For example, you can easily load your own dataset following [this tutorial](https://huggingface.co/docs/datasets/loading#local-and-remote-files).

More precisely, we are going to explain how to fine-tune the [Llama2 model](https://huggingface.co/docs/transformers/model_doc/llama2) on the [Europarl-ST dataset](https://huggingface.co/datasets/tj-solergibert/Europarl-ST), but only that [dataset of Europarl-ST focused on the text data for MT from English](https://huggingface.co/datasets/tj-solergibert/Europarl-ST-processed-mt-en).

In [2]:
from datasets import load_dataset

raw_datasets = load_dataset("tj-solergibert/Europarl-ST-processed-mt-en")

/home/jorcisai/Docencia/.venv_python3.10_keras/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


As shown, each English sentence is repeated for each of the seven target languages (0: 'de', 2: 'es', 3: 'fr', 4: 'it', 5: 'nl', 6: 'pl', 7: 'pt').

The Llama2 model is a pretrained Large Language Model (LLM) ready to tackle several NLP tasks, being one of the them the translation from English into Spanish. Let us filter the Europarl-ST only for English into Spanish using a simple [lambda function](https://realpython.com/python-lambda/) with the [Dataset.filter() function](https://huggingface.co/docs/datasets/v2.9.0/en/package_reference/main_classes#datasets.Dataset.filter) and taking a small sample with [Dataset.select() function](https://huggingface.co/docs/datasets/v2.9.0/en/package_reference/main_classes#datasets.Dataset.select). The reason to take a small sample is because of time and computational constraints.

In [3]:
lang="es"
random_seed = 23
max_source_test_len = 40
lang_id = raw_datasets["train"].features["dest_lang"].names.index(lang)
train_dataset = raw_datasets["train"].filter(lambda x: x["dest_lang"] == lang_id and len(x["source_text"])<max_source_test_len).shuffle(seed=random_seed).select(range(256))
dev_dataset   = raw_datasets["valid"].filter(lambda x: x["dest_lang"] == lang_id and len(x["source_text"])<max_source_test_len).shuffle(seed=random_seed).select(range(16))
test_dataset  = raw_datasets["test"].filter(lambda x: x["dest_lang"] == lang_id and len(x["source_text"])<max_source_test_len).shuffle(seed=random_seed).select(range(64))

Now we load the pre-trained tokenizer for the Llama2 model and apply it to a sample English-Spanish pair:

In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
from transformers import AutoTokenizer

max_tok_length = 50
checkpoint = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(
    checkpoint, use_auth_token=True,
    #padding=True,
    #pad_to_multiple_of=8,
    #truncation=True,
    #max_length=max_tok_length,
    padding_side='left',
    )
tokenizer.pad_token = "[PAD]"

/home/jorcisai/Docencia/.venv_python3.10_keras/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


We can apply the tokenizer function to any dataset taking advantage that Hugging Face Datasets are [Apache Arrow](https://arrow.apache.org) files stored on the disk, so you only keep the samples you ask for loaded in memory.

To keep the data as a dataset, we will use the [Dataset.map() function](https://huggingface.co/docs/datasets/en/package_reference/main_classes#datasets.Dataset.map). This also allows us some extra flexibility, if we need more preprocessing done than just tokenization. The map() method works by applying a function on each element of the dataset.

In our case, each sample pair is going to be preprocessed according to the training needs of the model that is to be finetuned. In the case of Llama2, it is recommended to explicitly state a task prompt for each source sentence:

In [10]:
src = "en"
tgt = lang
task_prefix = f"Translate from {src} to {tgt}:\n"
num_shots = 1
sample = train_dataset.shuffle(seed=random_seed).select(range(num_shots))
shots = ""
for s in sample: shots += f"en: {s['source_text']} = es: {s['dest_text']}\n"

def preprocess4test_function(batch):
    inputs = [f"{task_prefix}{shots}{src}: {s} = {tgt}: " for s in batch["source_text"]]
    model_inputs = tokenizer(inputs,padding=True,)
    return model_inputs

The way the Datasets library applies this processing is by adding new fields to the datasets, one for each key in the dictionary returned by the tokenize function, that is, *input_ids*, *attention_mask* and *labels*:

In [11]:
sample = test_dataset.select(range(2))
model_input = preprocess4test_function(sample)
print(model_input)
print(tokenizer.batch_decode(model_input.input_ids))

{'input_ids': [[1, 4103, 9632, 515, 427, 304, 831, 29901, 13, 264, 29901, 14933, 674, 11719, 5129, 694, 16156, 29889, 353, 831, 29901, 6274, 5826, 941, 9658, 9014, 20484, 859, 694, 9871, 13, 264, 29901, 15366, 310, 445, 2924, 29889, 353, 831, 29901, 29871], [1, 4103, 9632, 515, 427, 304, 831, 29901, 13, 264, 29901, 14933, 674, 11719, 5129, 694, 16156, 29889, 353, 831, 29901, 6274, 5826, 941, 9658, 9014, 20484, 859, 694, 9871, 13, 264, 29901, 910, 338, 263, 4472, 29889, 353, 831, 29901, 29871]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [12]:
preprocessed_test_dataset = test_dataset.map(preprocess4test_function, batched=True)

Map: 100%|██████████| 64/64 [00:00<00:00, 3002.16 examples/s]


In [14]:
for i in range(len(preprocessed_test_dataset['input_ids'])):
    print(preprocessed_test_dataset['input_ids'][i])
    print(preprocessed_test_dataset['attention_mask'][i])

[0, 0, 0, 0, 0, 0, 0, 1, 4103, 9632, 515, 427, 304, 831, 29901, 13, 264, 29901, 14933, 674, 11719, 5129, 694, 16156, 29889, 353, 831, 29901, 6274, 5826, 941, 9658, 9014, 20484, 859, 694, 9871, 13, 264, 29901, 15366, 310, 445, 2924, 29889, 353, 831, 29901, 29871]
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 1, 4103, 9632, 515, 427, 304, 831, 29901, 13, 264, 29901, 14933, 674, 11719, 5129, 694, 16156, 29889, 353, 831, 29901, 6274, 5826, 941, 9658, 9014, 20484, 859, 694, 9871, 13, 264, 29901, 910, 338, 263, 4472, 29889, 353, 831, 29901, 29871]
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 1, 4103, 9632, 515, 427, 304, 831, 29901, 13, 264, 29901, 14933, 674, 11719, 5129, 694, 16156, 29889, 353, 831, 29901, 6274, 5826, 941, 9658, 9014, 20484, 859, 694, 9871,

In [49]:
sample = raw_datasets["test"].shuffle(seed=random_seed).select(range(1))
model_input = tokenize_function(sample)
print(model_input)
print(tokenizer.batch_decode(model_input.input_ids))
print(tokenizer.batch_decode(model_input.labels))


{'input_ids': [[1, 4103, 9632, 515, 427, 304, 831, 29901, 13, 264, 29901, 1670, 526, 5065, 5362, 4225, 29889, 353, 831, 29901, 11389, 443, 294, 16632, 7305, 5065, 29887, 5326, 29889, 13, 427, 29901, 15366, 310, 445, 2924, 29889, 353, 831, 29901, 29871]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[1, 9241, 5760, 5264, 712, 25725, 273, 29889]]}
['<s> Translate from en to es:\nen: There are urgent needs. = es: Hay unas necesidades urgentes.\n en: measures of this kind. = es: ']
['<s> resistencia social que provocan.']


bitsandbytes is a quantization library with a Transformers integration. With this integration, you can quantize a model to 8 or 4-bits and enable many other options by configuring the BitsAndBytesConfig class. For example, you can:

<ul>
<li>set load_in_4bit=True to quantize the model to 4-bits when you load it</li>
<li>set bnb_4bit_quant_type="nf4" to use a special 4-bit data type for weights initialized from a normal distribution</li>
<li>set bnb_4bit_use_double_quant=True to use a nested quantization scheme to quantize the already quantized weights</li>
<li>set bnb_4bit_compute_dtype=torch.bfloat16 to use bfloat16 for faster computation</li>
</ul>


In [15]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

Pass the quantization_config to the from_pretrained method.

In [16]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    quantization_config=quantization_config,
    torch_dtype=torch.bfloat16,
)


/home/jorcisai/Docencia/.venv_python3.10_keras/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.30s/it]
/home/jorcisai/Docencia/.venv_python3.10_keras/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Inference

Loading default inference parameters for the model, so that additional parameters could be added and passed to the [generate function](https://huggingface.co/docs/transformers/main_classes/text_generation):

In [17]:
from transformers import GenerationConfig

generation_config = GenerationConfig.from_pretrained(
    checkpoint,
    )

print(generation_config)

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 0,
  "temperature": 0.6,
  "top_p": 0.9,
  "transformers_version": "4.30.0"
}



Running inference with maximum generation length set to max_dest_length, otherwise default value is 4096:

In [19]:
test_batch_size = 4
batch_tokenized_test = preprocessed_test_dataset.batch(test_batch_size)

Batching examples: 100%|██████████| 64/64 [00:00<00:00, 1015.33 examples/s]


In [30]:
number_of_batches = len(batch_tokenized_test["input_ids"])
output_sequences = []
for i in range(number_of_batches):
    output_batch = model.generate(generation_config=generation_config, input_ids=torch.tensor(batch_tokenized_test["input_ids"][i]).cuda(), attention_mask=torch.tensor(batch_tokenized_test["attention_mask"][i]).cuda(), max_length = max_tok_length+20, num_beams=1, do_sample=False,)
    output_sequences.extend(output_batch)

## Evaluation

The last thing to define for our Trainer is how to compute the metrics to evaluate the predictions of our model with respect to references. To this purpose, we use the [Evaluate library](https://huggingface.co/docs/evaluate) which includes the definition of generic and task-specific metrics. In our case, we use the [BLEU metric](https://huggingface.co/spaces/evaluate-metric/bleu), or to be more precise, [sacreBLEU](https://huggingface.co/spaces/evaluate-metric/sacrebleu). You can see a simple example of usage below:

In [21]:
from evaluate import load

metric = load("sacrebleu")

The example below performs a basic post-processing to decode the predictions and extract the translation:

In [28]:
import re

def compute_metrics(sample, output_sequences):
    inputs = [f"{task_prefix}{shots}{src}: {s} = {tgt}: " for s in sample["source_text"]]
    preds = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)
    print(inputs)
    print(preds)
    for i, (input,pred) in enumerate(zip(inputs,preds)):
      pred = re.search(r'^.*\n',pred.removeprefix(input).strip())
      if pred is not None:
        preds[i] = pred.group()[:-1]
      else:
        preds[i] = ""
    print(sample["source_text"])
    print(sample["dest_text"])
    print(preds)
    result = metric.compute(predictions=preds, references=sample["dest_text"])
    result = {"bleu": result["score"]}
    return result

In [31]:
result = compute_metrics(preprocessed_test_dataset,output_sequences)
print(f'BLEU score: {result["bleu"]}')

['Translate from en to es:\nen: Britain will vote ‘ no ’. = es: Gran Bretaña votará « no ».\nen: measures of this kind. = es: ', 'Translate from en to es:\nen: Britain will vote ‘ no ’. = es: Gran Bretaña votará « no ».\nen: This is a template. = es: ', 'Translate from en to es:\nen: Britain will vote ‘ no ’. = es: Gran Bretaña votará « no ».\nen: That is why this issue is so relevant. = es: ', 'Translate from en to es:\nen: Britain will vote ‘ no ’. = es: Gran Bretaña votará « no ».\nen: Now it is the turn of the constitution. = es: ', 'Translate from en to es:\nen: Britain will vote ‘ no ’. = es: Gran Bretaña votará « no ».\nen: It must be flexible, but it must exist. = es: ', 'Translate from en to es:\nen: Britain will vote ‘ no ’. = es: Gran Bretaña votará « no ».\nen: It has become STX of Korea. = es: ', 'Translate from en to es:\nen: Britain will vote ‘ no ’. = es: Gran Bretaña votará « no ».\nen: Let us reinvigorate our values. = es: ', 'Translate from en to es:\nen: Britain wil